# The Python compile module

In [8]:
    import ast, sys
    from json import load
    from pathlib import Path
    from textwrap import dedent
    from codeop import Compile

In [9]:
    class Compiler(Compile):
        """{Shell} provides the IPython machinery to objects."""
        def ast_transform(Compiler, node): return node
        
        @property
        def transform(Compiler): return dedent

        def compile(Compiler, ast): return compile(ast, Compiler.filename, 'exec')
                
        def ast_parse(Compiler, source, filename='<unknown>', symbol='exec', lineno=0): 
            return ast.increment_lineno(ast.parse(source, Compiler.filename, 'exec'), lineno)

In [3]:
    class NotebookExporter:
        def from_file(self, file_stream, resources=None, **dict):
            return self.from_notebook_node(load(file_stream), resources=resources, **dict)
        
        def from_filename(self, filename, resources=None, **dict):
            with open(filename, 'r') as file_stream:
                return self.from_file(file_stream, resources, **dict)
            
        def from_notebook_node(self, nb, resources=None, **dict): 
            return nb, resources

In [4]:
    class PythonExporter(NotebookExporter):
        def from_notebook_node(self, nb, resources=None,  **dict):
            nb, resources = super().from_notebook_node(nb, resources, **dict)
            for i, cell in enumerate(nb['cells']):
                if isinstance(cell['source'], list):
                    nb['cells'][i]['source'] = ''.join(cell['source'])
                
            return ("\n"*2).join(
                dedent(cell['source']) for cell in nb['cells'] if cell['cell_type'] == 'code'
            ), resources
            

In [5]:
    NotebookNode = dict

In [6]:
    if __name__ ==  '__main__':
        try:  from .compile import export
        except: from compile import export
        export('compile_python.ipynb', '../importnb/compile_python.py')
        __import__('doctest').testmod()